#ImgTools
Set of tools for the image processing of Escher transitions.
##Version History
###V2
* Changed to vertical letters.

###V1
* Moved 'findmidpoints' from 'Escher Tools' to 'Img Tools'.

###V0
* Initial release.
* Mostly copied from 'Trace.jl' with some functions given more descriptive names.
* Added 'padwhite' and 'invert'.

In [1]:
using Images

In [2]:
function gray2bw(img, threshold=0.5)
    #converts grayscale image to BW
    
    whitepixels = img .> threshold #find white pixels
    bw = zeros(img) #intialize to black
    bw[whitepixels] = one(img[1,1]) #write white pixels
    
    return bw
end

gray2bw (generic function with 2 methods)

In [3]:
function stripwhite(x)
    #removes any first or last rows or columns that are entirely less than 0.5

    while sum(x[:,1] .< .5) == 0 
        x = x[:,2:end]
    end
    
    while sum(x[:,end] .< .5) == 0
        x = x[:,1:end-1]
    end
    
    while sum(x[1,:] .< .5) == 0
        x = x[2:end,:]
    end
    
    while sum(x[end,:] .< .5) == 0
        x = x[1:end-1,:]
    end
    
    return x
    
end

stripwhite (generic function with 1 method)

In [4]:
function padblack(x)
    #adds a frame of zeros around x
    
    black = zero(x[1,1])
    
    x = cat(2,fill(black,size(x)[1]),x,fill(black,size(x)[1]))
    return cat(1,fill(black,size(x)[2])',x,fill(black,size(x)[2])')
    
end

padblack (generic function with 1 method)

In [5]:
function padwhite(x, thickness)
    #adds a frame of 'thickness' of ones around 'x'
    
    white = one(x[1,1])
    
    x = cat(2,fill(white,(size(x)[1], thickness)),x,fill(white,(size(x)[1]),thickness))
    return cat(1,fill(white,(thickness, size(x)[2])),x,fill(white,(thickness, size(x)[2])))
    
end

padwhite (generic function with 1 method)

In [32]:
function prepimg(file, whiteratio = .2)
    #preps image located at 'file'
    
    img = imread(file) #load
    img = gray2bw(img) #make black and white
    img = stripwhite(img) #trim excess white space
    thickness = iround(whiteratio * size(img)[1]) #how much border to add
    img = padwhite(img, thickness) #add white border
    img = padblack(img) #add black border
    
    imwrite(img',file[1:end-4]*".bmp") #save
    
    return img
    
end

prepimg (generic function with 2 methods)

In [8]:
function edgedetect(img)
    #traces edges of monochrome bmp at 'filename' from first white pixel
    
    white = one(img[1,1])
    black = zero(img[1,1])
    loop = 0 #loop counter
    loc = [(-1,-1)  (0,-1)  (1,-1)  (1,0)  (1,1)  (0,1)  (-1,1)  (-1,0)] #neighbors relative to center
    
    #find first white pixel
    start = findfirst(img, white)
    start = ind2sub(size(img),start) #convert to location
    
    x = [start[1]]
    y = [start[2]]
    path = 4
    
    while true #loop escape is as return at bottom
        #trace edge
        
        #"unroll" neighboring pixels
        neighbors = [img[x[end]-1,y[end]-1],img[x[end],y[end]-1]]
        push!(neighbors, img[x[end]+1,y[end]-1])
        push!(neighbors, img[x[end]+1,y[end]])
        push!(neighbors, img[x[end]+1,y[end]+1])
        push!(neighbors, img[x[end],y[end]+1])
        push!(neighbors, img[x[end]-1,y[end]+1])
        push!(neighbors, img[x[end]-1,y[end]])   
        
        #find path
        nextblack = findnext(neighbors, black, mod(path-5,8)+1) #gives preference to hard left turn
        if nextblack == 0
            nextblack = findfirst(neighbors, black)
            if nextblack == 0
                error("no black neighbors")
            end
        end

        path = findnext(neighbors, white, mod(nextblack,8)+1) #find path
        if path == 0 
            path = findfirst(neighbors, white)
            if path == 0
                error("no white neighbors")
            end
        end
        
        push!(x, x[end]+loc[path][1]) #update path           
        push!(y, y[end]+loc[path][2])
        
        if (x[end],y[end]) == (x[1],y[1]) #returned to begining
            return x,y
        end
        
        loop += 1 #increment loop counter
        if loop > 1e6 #escape after number of iterations for testing/safety
            if size(hcat(x,y)) == size(unique(hcat(x,y),1))
                loop = 0
            else
                warn("trapped in loop?")
                return x,y
            end
        end
    end
end

edgedetect (generic function with 1 method)

In [9]:
function invert(img)
    #converts black to white and white to black
    
    newblacks = (img .== one(img[1,1])) #find white pixels
    inverse = ones(img) #intialize to white
    inverse[newblacks] = zero(img[1,1])
    
    return inverse
end    

invert (generic function with 1 method)

In [10]:
function findmidpoints(x, dim = 2)
    #Returns the center point (in terms of 'dim') of each chunk of entirely white (or black) rows
    
    blackpixels = x .< 0.5 #find black pixels
    whiterow = sum(blackpixels, dim) .<= 2 #rows that are all white (except for border)
    
    whitemidpoints = Int[] #midpoints of spaces
    blackmidpoints = Int[2] #midpoints of letters
    ind = 2 #start inside border
    ind2 = 2 #irrelevant start point, will be thrown out
    
    while true #escape in return statement
        ind = findnext(whiterow,false,ind) #find next non-white row (start of letter)
        
        push!(whitemidpoints, ifloor((ind-ind2)/2)+ind2) #add midpoint of space to list
        
        ind2 = findnext(whiterow,true,ind) #find next white row (end of letter)
        
        if ind2 != 0
            push!(blackmidpoints, ifloor((ind2-ind)/2)+ind) #add midpoint of letter to list
        else
            whitemidpoints[1] = 2 #start of image
            whitemidpoints[end] = size(x)[2]-1 #end of image
            push!(blackmidpoints, size(x)[2]-1)
            return whitemidpoints, blackmidpoints #throw away space at begining of word
        end
            
        ind = ind2 #search from end of last letter
        
    end
end

findmidpoints (generic function with 2 methods)